In [ ]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt

warnings.simplefilter('ignore')

In [ ]:
metro_file = './202504-Nakamozu-OD.csv'
weather_file = './sakai_weather.csv'

df_metro = pd.read_csv(metro_file, encoding="shift-jis")
df_weather = pd.read_csv(weather_file, encoding="shift-jis")

#index追加
df_metro['data_number'] = df_metro.index

#:00を除去
df_metro['arrival_station_time'] = df_metro['arrival_station_time'].str.replace(':00', '')
df_metro['depature_station_time'] = df_metro['depature_station_time'].str.replace(':00', '')

#日のみをintで表記+:00の辻褄合わせ
date_iter = 1
while(date_iter <= 30):
    date_str = str(date_iter)
    
    if (date_iter < 10):
        date_str = '0' + date_str
    
    #辻褄合わせで使用
    time_str = date_str
    
    date_str = '2025-04-' + date_str

    df_metro.replace(date_str, date_iter, inplace=True)
    
    #辻褄合わせ
    if (date_iter <= 24):
        df_metro.replace(time_str, time_str + ':00', inplace=True)

    date_iter += 1

In [ ]:
#なかもずから出発したものだけを抽出
df_depature = df_metro[df_metro['depature_station'] == 'なかもず']

In [ ]:
#t-1時30分～t時29分まではt時の天気を参照にするため、参照時間「rainy_time」を追加
hour_iter = 1
list_num = 0

dflist_for_rainy_time = []
dflist_for_rainy_time.append(pd.DataFrame(df_depature['depature_station_time'].str.contains('初期値を入力しているだけです')))
while(hour_iter <= 24):
    dflist_for_rainy_time.append(pd.DataFrame(df_depature['depature_station_time'].str.contains('初期値を入力しているだけです')))

    hour_str = str(hour_iter) + ':'
    
    if (hour_iter < 10):
        hour_str = '0' + hour_str

    for ten_minute in range(6):
        ten_minute_str = str(ten_minute)
        use_hour_str = hour_str + ten_minute_str

        #30分を越したら次の時間に保存場所を切り替える
        if (ten_minute == 3):
            dflist_for_rainy_time[list_num].replace(True, hour_iter, inplace=True)
            dflist_for_rainy_time[list_num].replace(False, 0, inplace=True)

            list_num += 1

        dflist_for_rainy_time[list_num]['depature_station_time'] += df_depature['depature_station_time'].str.contains(use_hour_str)

    
    if (hour_iter == 24):
        dflist_for_rainy_time[list_num].replace(True, hour_iter, inplace=True)
        dflist_for_rainy_time[list_num].replace(False, 0, inplace=True)
           

    hour_iter += 1
    

In [ ]:
#先ほどのリストをdf_departureに取り込み
check = 0
for df_temporary in dflist_for_rainy_time:
    if (check == 0):
        df_depature['rainy_time'] = df_temporary['depature_station_time']

    df_depature['rainy_time'] += df_temporary['depature_station_time']

    check += 1

In [ ]:
#日ごとに分けてリストに格納(日ごとにわける弊害で0のrainy_timeが参照不可)
df_depature_datelist = []

date_iter = 1
while (date_iter <= 30):
    df_depature_datelist.append(df_depature[df_depature['data_date'] == date_iter])

    date_iter += 1

In [ ]:
#rainy_time毎の利用者数をdf_weatherに記録(rainy_timeが0のものは省かれている)
weather_index = 0
day = -1
while (weather_index < len(df_weather)):
    time = weather_index % 24 + 1
    if (time == 1):
        day += 1
    
    size = len(df_depature_datelist[day][df_depature_datelist[day]['rainy_time'] == time])

    df_weather.iat[weather_index, 5] = size

    weather_index += 1

#columns書き換え
df_weather.rename(columns={'均質番号':'降水量を参考にした利用者数'}, inplace=True)

以下は、プロットとデータのcsv化のコード

In [ ]:
#プロット用の「日時」作成
df_weather['日時'] = df_weather['日'].astype(str) + '-' + df_weather['時'].astype(str)

In [ ]:
#df_weatherをプロット(全体見る用)
#降水量があるところは青それ以外は緑
colors = df_weather['降水量(mm)'].apply(lambda x: 'blue' if (x > 0) else 'green')

#プロット
plt.figure(figsize=(30,8))
plt.bar(df_weather['日時'], df_weather['降水量を参考にした利用者数'], color=colors)
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
plt.xlabel('')
plt.ylabel('')
plt.title('')
plt.show()

In [ ]:
#df_weatherをプロット(一部可視化その1)
mask1 = (df_weather['日'] >= 21) & (df_weather['日'] <= 25)
df_weather_filtered1 = df_weather.loc[mask1]

#降水量があるところは青それ以外は緑
colors = df_weather_filtered1['降水量(mm)'].apply(lambda x: 'blue' if (x > 0) else 'green')

plt.figure(figsize=(30,8))
plt.xticks(rotation=45, ha='right')
plt.bar(df_weather_filtered1['日時'], df_weather_filtered1['降水量を参考にした利用者数'], color=colors)
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
#df_weatherをプロット(一部可視化その2)
mask2 = (df_weather['日'] >= 7) & (df_weather['日'] <= 11)
df_weather_filtered2 = df_weather.loc[mask2]

#降水量があるところは青それ以外は緑
colors = df_weather_filtered2['降水量(mm)'].apply(lambda x: 'blue' if (x > 0) else 'green')

plt.figure(figsize=(30,8))
plt.xticks(rotation=45, ha='right')
plt.bar(df_weather_filtered2['日時'], df_weather_filtered2['降水量を参考にした利用者数'], color=colors)
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
#df_weatherをプロット(一部可視化その3)
mask3 = ((df_weather['日'] % 7) == 4)
df_weather_filtered3 = df_weather.loc[mask3]

#降水量があるところは青それ以外は緑
colors = df_weather_filtered3['降水量(mm)'].apply(lambda x: 'blue' if (x > 0) else 'green')

plt.figure(figsize=(30,8))
plt.xticks(rotation=45, ha='right')
plt.bar(df_weather_filtered3['日時'], df_weather_filtered3['降水量を参考にした利用者数'], color=colors)
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
#df_weatherをプロット(一部可視化その4)
mask4 = ((df_weather['日'] % 7) == 6)
df_weather_filtered4 = df_weather.loc[mask4]

#降水量があるところは青それ以外は緑
colors = df_weather_filtered4['降水量(mm)'].apply(lambda x: 'blue' if (x > 0) else 'green')

plt.figure(figsize=(30,8))
plt.xticks(rotation=45, ha='right')
plt.bar(df_weather_filtered4['日時'], df_weather_filtered4['降水量を参考にした利用者数'], color=colors)
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
#降水量がある部分のデータを抽出
df_rainy = df_weather[df_weather['降水量(mm)'] > 0]
df_rainy.reset_index(inplace=True, drop=True)

In [ ]:
#df_weatherをcsvファイルとして取り出す
df_weather.to_csv('sakai_weather_with_usernum.csv', index=False)

In [ ]:
#df_rainyをcsvファイルとして取り出す
df_rainy.to_csv('sakai_rainy_with_user.csv', index=False)

# 以下は試行錯誤で使ったセル

In [ ]:
df_weather.head(10)

In [ ]:
print(df_depature_datelist[0][df_depature_datelist[0]['rainy_time'] == 0])

In [ ]:
dflist_for_rainy_time[23][dflist_for_rainy_time[7]['depature_station_time'] == 22].head(10)

In [ ]:
dflist_for_rainy_time[8].head(10)

In [ ]:
for i in range(len(dflist_for_rainy_time)):
    print(len(dflist_for_rainy_time[i]))

In [ ]:
print(df_depature['depature_station_time'].str.contains('10:2') + df_depature['depature_station_time'].str.contains('10:3'))

In [ ]:
df_depature.head(25)

In [ ]:
#日ごとに分けてリストに格納
df_depature_datelist = []

date_iter = 1
while (date_iter <= 30):
    df_depature_datelist.append(df_depature[df_depature['data_date'] == date_iter])

    date_iter += 1

In [ ]:
df_tameshi = df_depature_datelist[0]

df_tameshi['rainy_time'] = df_tameshi['depature_station_time'].str.contains('16:23')
df_tameshi2 = df_tameshi['depature_station_time'].str.contains('16:22')
df_tameshi.replace(True, 15, inplace=True)
df_tameshi.replace(False, 0, inplace=True)

#df_tameshi2.replace(True, 15, inplace=True)
#df_tameshi2.replace(False, 0, inplace=True)

df_tameshi['rainy_time'] += df_tameshi2

In [ ]:
df_tameshi['rainy_time'] += df_tameshi2

In [ ]:
df_tameshi.head(25)

In [ ]:
for i in range(len(df_depature_datelist)):
    print(i+1, len(df_depature_datelist[i]))

In [ ]:
df_depature_datelist[29].head(25)

In [ ]:
df_rainy = df_weather[df_weather['降水量(mm)'] > 0]
df_rainy.reset_index(inplace=True, drop=True)

In [ ]:
print(df_rainy.loc[0, ['日', '時']].tolist())

In [ ]:
print(df_rainy.loc[:, ['日', '時']])

In [ ]:
df_rainy.head(40)

In [ ]:
print(type(df_metro.loc[0]['depature_station_time']))
print(type(df_weather.loc[0]['降水量(mm)']))

In [ ]:
df_metro.head(10)

In [ ]:
print(df_weather[df_weather['日'] == 2])

In [ ]:
df_weather.head(10)